In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame(pd.read_csv('std_data_5min.csv'))
PReal = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_realpower"]
Pust = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_ust_predpower"]


# 构造10min考核点
# 由5min构造10min考核点时，由于是后向平均，需要用i点和i点前一个数据求和除以二
# 105119
PReal = np.array(PReal)  # 确保PReal是一个numpy数组
averagesPReal = (PReal[:-1] + PReal[1:]) / 2.0
averagesPReal = np.append(averagesPReal, averagesPReal[-1]) # 保持数组长度一致

# 计算相邻10分钟的功率波动
fluctuations = []
for i in range(0, len(averagesPReal) - 2):  # 减2是因为我们是计算相隔10分钟（即每隔一个索引）的差值
    fluctuation = abs(averagesPReal[i] - averagesPReal[i+2])
    fluctuations.append(fluctuation)


df["realpower_Fluctuation"] = pd.DataFrame(fluctuations)



将df["realpower_Fluctuation"]这一列的波动做一个判断，判断每一天有多少个数据超过了0.02218，并且用go库画一个二维图，显示365天，每一天有多少数据超过了0.02218

In [2]:
import pandas as pd
import plotly.graph_objects as go



df['Datetime'] = pd.to_datetime(df['Datetime'])

# 计算每一天超过0.02218波动的数据点数量
df['date'] = df['Datetime'].dt.date  # 创建一个新列，只包含日期
threshold = 0.02218 # 标幺化后的50MW，即本文电场装机容量是2254.4MW，大于150MW，应按照表第二列50MW考核规则考核。对50MW除以2254.4MW进行标准化，得到0.02218，当有功波动超过0.02218时，要依据细则进行考核。

# 使用groupby按照日期分组，并对每组使用lambda函数计算超过阈值的数量
daily_over_threshold = df.groupby('date')['realpower_Fluctuation'].apply(lambda x: (x > threshold).sum()).reset_index(name='count_over_threshold')

# 使用plotly绘制结果
fig = go.Figure(data=go.Scatter(x=daily_over_threshold['date'], y=daily_over_threshold['count_over_threshold'], mode='lines+markers'))

# 设置图表的标题和轴标签
fig.update_layout(title='每日超过0.02218功率波动的数据点数量',
                  xaxis_title='日期',
                  yaxis_title='超过阈值的数据点数量',
                   font=dict(
                      size=15  # 设置字体大小为14
                  ))

# 显示图表
fig.show()


In [5]:
import pandas as pd
import plotly.graph_objects as go

# 假设daily_over_threshold是包含每天日期和超过阈值的数据点数量的DataFrame
window_size = 90  # 举例：使用7天的移动平均

# 计算移动平均
daily_over_threshold['moving_average'] = daily_over_threshold['count_over_threshold'].rolling(window=window_size, min_periods=1).mean()

# 绘制原始数据点
fig = go.Figure(data=go.Scatter(x=daily_over_threshold['date'], y=daily_over_threshold['count_over_threshold'], mode='lines+markers', name='Daily Count'))

# 绘制趋势线（移动平均）
fig.add_trace(go.Scatter(x=daily_over_threshold['date'], y=daily_over_threshold['moving_average'], mode='lines', name='Trend (Moving Average)'))

# 设置图表的标题和轴标签
fig.update_layout(title='每日超过0.02218功率波动的数据点数量与趋势',
                  xaxis_title='日期',
                  yaxis_title='超过阈值的数据点数量')

# 显示图表
fig.show()


# 美化图片适用于文章

In [4]:
import pandas as pd
import plotly.graph_objects as go

# 假设daily_over_threshold是包含每天日期和超过阈值的数据点数量的DataFrame
window_size = 90  # 举例：使用7天的移动平均

# 计算移动平均
daily_over_threshold['moving_average'] = daily_over_threshold['count_over_threshold'].rolling(window=window_size, min_periods=1).mean()

# 绘制原始数据点
fig = go.Figure(data=go.Scatter(x=daily_over_threshold['date'], y=daily_over_threshold['count_over_threshold'], mode='markers', name='Daily Count'))

# 绘制趋势线（移动平均）
fig.add_trace(go.Scatter(x=daily_over_threshold['date'], y=daily_over_threshold['moving_average'], mode='lines', name='Trend (Moving Average)'))

# 设置图表的标题和轴标签，并调整字体大小
fig.update_layout(title='每日超过0.02218功率波动的数据点数量与趋势({}日)'.format(window_size),
                  xaxis_title='日期',
                  yaxis_title='超过阈值的数据点数量',
                  font=dict(
                      size=14  # 设置字体大小为14
                  ),
                  legend=dict(
                      x=0.82,  # 控制图例在x轴上的位置
                      y=0.95   # 控制图例在y轴上的位置
                  ))

# 显示图表
fig.show()
